In [1]:
%matplotlib inline
import sys
import time
import math
import copy
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as sf

from lightfm import LightFM
import lightfm.evaluation
import scipy.sparse as sparse
from scipy.special import expit
from pyspark.sql.types import *
from skopt import forest_minimize

plt.style.use('bmh')
pd.options.display.max_columns = 500

from sklearn.feature_extraction import DictVectorizer

/opt/conda/anaconda/lib/python2.7/site-packages/sklearn/utils/__init__.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/opt/conda/anaconda/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/opt/conda/anaconda/lib/python2.7/site-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/opt/conda/anaconda/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/opt/conda/anaconda/lib/python2.7/site-packages/sklearn/metrics/pairwis

## Data

In [2]:
days = 22

paths = []
for i in range(1, 23):
    paths.append('gs://dataproc-jupyter-eileen.npo-data.nl/data/events/march/_day=' + str(i))
interactions = spark.read.parquet(*paths)
print(interactions.count())
interactions.printSchema()

df_int = interactions.toPandas()
df_int = df_int.rename(columns={"seriesRef":"mid", "npoprofileid":"uid"})

df_cont = spark.read.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/poms_stream/").select('seriesRef').dropDuplicates().toPandas()
df_cont = df_cont.rename(columns={"seriesRef": "mid"})

# add together df_int['mid'] and df_cont['mid'] remove duplicates
df_cont = pd.DataFrame(pd.concat([df_cont['mid'], df_int['mid']])).drop_duplicates()

n_users = df_int.uid.unique().shape[0]
n_items = df_cont.mid.unique().shape[0]
sparsity = float(df_int.shape[0]) / float(n_users*n_items) * 100
print('Threshold - Starting interactions info')
print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(sparsity))

# # Thresholding interactions based on user
# def threshold_interactions(df, df_cont, mid_min):
#     n_users = df.uid.unique().shape[0]
#     n_items = df_cont.mid.unique().shape[0]
#     sparsity = float(df_int.shape[0]) / float(n_users*n_items) * 100
#     print('Threshold - Starting interactions info')
#     print('Number of users: {}'.format(n_users))
#     print('Number of models: {}'.format(n_items))
#     print('Sparsity: {:4.3f}%'.format(sparsity))
    
#     mid_counts = df.groupby('uid').mid.count()
#     df = df[~df.uid.isin(mid_counts[mid_counts < mid_min].index.tolist())]
    
#     n_users = df.uid.unique().shape[0]
#     n_items = df_cont.mid.unique().shape[0]
#     sparsity = float(df_int.shape[0]) / float(n_users*n_items) * 100
#     print('Threshold - Ending interactions info')
#     print('Number of users: {}'.format(n_users))
#     print('Number of models: {}'.format(n_items))
#     print('Sparsity: {:4.3f}%'.format(sparsity))
#     return df

#df_lim = threshold_interactions(df_int, df_cont, 5) 
df_lim = df_int

# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df_cont.mid.unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_lim.uid.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid
    
def map_ids(row, mapper):
    return mapper[row]

I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).values
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).values
V = np.ones(I.shape[0])
interactions = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
interactions = interactions.tocsr()

interactions

1235728
root
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- npoprofileid: string (nullable = true)
 |-- seriesRef: string (nullable = true)

Threshold - Starting interactions info
Number of users: 181357
Number of models: 3169
Sparsity: 0.215%


<181357x3169 sparse matrix of type '<type 'numpy.float64'>'
	with 674689 stored elements in Compressed Sparse Row format>

## Train-test split

In [3]:
# For all watched stuff
train_day = df_int[df_int['day']!=days]
train_day = train_day[train_day['uid'].isin(df_lim.uid.tolist())]
test_day = df_int[df_int['day']==days]
test_day = test_day[test_day['uid'].isin(df_lim.uid.tolist())]

intersect = np.intersect1d(train_day.uid.unique(), test_day.uid.unique())
trainday = train_day[train_day['uid'].isin(intersect)].copy()
testday = test_day[test_day['uid'].isin(intersect)].copy()

# map users and items to idx
user_list = []
item_list = []
for idx, row in testday.iterrows():
    idx_user = uid_to_idx.get(row.uid)
    idx_item = mid_to_idx.get(row.mid)
    user_list.append(idx_user)
    item_list.append(idx_item)
testday['user'] = user_list
testday['item'] = item_list

def train_test_split(ratings):
    train = ratings.copy().tocoo()
    print(train.shape)
    test = sparse.lil_matrix(train.shape) 
    user_index = testday.user.tolist()        
    train = train.tolil()

    for user in user_index:
        test_ratings = testday[testday['user'] == user].item.tolist()
        test[user, test_ratings] = ratings[user, test_ratings]
    return train.tocsr(), test.tocsr(), user_index

train, test, user_index = train_test_split(interactions)

eval_train = train.copy()
non_eval_users = list(set(range(train.shape[0])) - set(user_index))

eval_train = eval_train.tolil()
for u in non_eval_users:
    eval_train[u, :] = 0.0
eval_train = eval_train.tocsr()

(181357, 3169)


In [4]:
# For recommended and watched stuff
recwatched_df = spark.read.parquet('gs://dataproc-jupyter-eileen.npo-data.nl/data/events/recwatched').toPandas()
recwatched_df = recwatched_df.rename(columns={"seriesRef":"mid", "npoprofileid":"uid"})
recwatched_df

train_day2 = train_day
test_day2 = recwatched_df[recwatched_df['uid'].isin(df_lim.uid.tolist())]

intersect2 = np.intersect1d(train_day2.uid.unique(), test_day2.uid.unique())
trainday2 = train_day2[train_day2['uid'].isin(intersect2)].copy()
testday2 = test_day2[test_day2['uid'].isin(intersect2)].copy()

# map users and items to idx
user_list2 = []
item_list2 = []
for idx, row in testday2.iterrows():
    idx_user = uid_to_idx.get(row.uid)
    idx_item = mid_to_idx.get(row.mid)
    user_list2.append(idx_user)
    item_list2.append(idx_item)
testday2['user'] = user_list2
testday2['item'] = item_list2

def train_test_split2(ratings):
    train2 = ratings.copy().tocoo()
    print(train2.shape)
    test2 = sparse.lil_matrix(train2.shape) 
    user_index2 = testday2.user.tolist()        
    train2 = train2.tolil()

    for user in user_index2:
        test_ratings2 = testday2[testday2['user'] == user].item.tolist()
        test2[user, test_ratings2] = ratings[user, test_ratings2]
    return train2.tocsr(), test2.tocsr(), user_index2

train2, test2, user_index2 = train_test_split2(interactions)

eval_train2 = train2.copy()
non_eval_users2 = list(set(range(train2.shape[0])) - set(user_index2))

eval_train2 = eval_train2.tolil()
for u in non_eval_users2:
    eval_train2[u, :] = 0.0
eval_train2 = eval_train2.tocsr()

(181357, 3169)


In [5]:
print("All watched " + str(len(testday)))
print("All rec and watched " + str(len(testday2)))

All watched 41358
All rec and watched 149


## The execution of the model

In [6]:
features = ['broadcaster', 'credits', 'description_tfidf', 'genres', 'title_tfidf', 'sub_tfidf']

def combinations(features):
    combi = []
    for i in range(1, len(features)+1):
        combi = combi + list(itertools.combinations(features,i))
    return combi

combi = combinations(features)
combi.insert(0, None)
print(len(combi))

def get_itemfeatures(sideinfo):
    feat_dlist = [{} for _ in idx_to_mid]
    for idx, row in sideinfo.iterrows():
        feat_key = '{}'.format(str(row.value).lower())
        idx = mid_to_idx.get(row.mid)
        if idx is not None:
            feat_dlist[idx][feat_key] = 1
    
    #print(feat_dlist[0])
    #print(len(feat_dlist))
    
    item_features = DictVectorizer().fit_transform(feat_dlist)
    #print(item_features.check_format)
    
    # Need to hstack item_features
    eye = sparse.eye(item_features.shape[0], item_features.shape[0]).tocsr()
    item_features_concat = sparse.hstack((eye, item_features))
    item_features_concat = item_features_concat.tocsr().astype(np.float32)
    
    return item_features_concat

def learning_curve(model, train, test, eval_train, train2, test2, eval_train2,
                        iterarray, user_features=None,
                        item_features=None, k=5,
                        **fit_params):
    old_epoch = 0
    test_patk = []
    test_patk_sd = []
    test2_patk = []
    test2_patk_sd = []
    test_rr = []
    test_rr_sd = []
    test2_rr = []
    test2_rr_sd = []
    for epoch in iterarray:
        print(epoch)
        more = epoch - old_epoch
        model.fit_partial(train, user_features=user_features,
                          item_features=item_features,
                          epochs=more, **fit_params)
        this_test_pk = lightfm.evaluation.precision_at_k(model, test, item_features=item_features, train_interactions=None, k=k)
        this_test_pk2 = lightfm.evaluation.precision_at_k(model, test2, item_features=item_features, train_interactions=None, k=k)
        this_test_rr = lightfm.evaluation.reciprocal_rank(model, test, item_features=item_features, train_interactions=None)
        this_test_rr2 = lightfm.evaluation.reciprocal_rank(model, test2, item_features=item_features, train_interactions=None)
        
        test_patk.append(np.mean(this_test_pk))
        test_patk_sd.append(np.std(this_test_pk))
        test2_patk.append(np.mean(this_test_pk2))
        test2_patk_sd.append(np.std(this_test_pk2))
        test_rr.append(np.mean(this_test_rr))
        test_rr_sd.append(np.std(this_test_rr))
        test2_rr.append(np.mean(this_test_rr2))
        test2_rr_sd.append(np.std(this_test_rr2))
        row = [epoch, test_patk[-1], test_patk_sd[-1], test2_patk[-1], test2_patk_sd[-1], test_rr[-1], test_rr_sd[-1], test2_rr[-1], test2_rr_sd[-1]]
        results.append(row)
        #precision.append([float(i) for i in list(this_test)])
        #rank.append([float(i) for i in list(this_test2)])
    return model, test_patk, test_patk_sd, test2_patk, test2_patk_sd, test_rr, test_rr_sd, test2_rr, test2_rr_sd

64


In [18]:
# don't touch plz
results = []
#precisions = []
#ranks = []

In [23]:
for j in range (50, 64):
    print(time.asctime()) 
    feature = combi[j]
    print(feature, j)

    if feature == None: 
        item_features_concat = None
    else: 
        paths = []
        for i in range(0, len(feature)):
            paths.append('gs://dataproc-jupyter-eileen.npo-data.nl/data/content_features/' + feature[i])
        sideinfo = spark.read.parquet(*paths)

        sideinfo = sideinfo.toPandas()
        sideinfo['value'] = sideinfo.value.str.encode('utf-8')
        sideinfo = sideinfo[sideinfo['mid'].isin(df_lim.mid.tolist())]
        sideinfo = sideinfo[sideinfo.groupby('value').value.transform(len) > 1] #remove unique content featuers
        item_features_concat = get_itemfeatures(sideinfo)
    
    result = []
#     precision = []
#     rank = []
    # Initialize model
    model = LightFM(loss='warp', random_state=2016)
    model.fit(train, item_features=item_features_concat, epochs=0)

    iterarray = range(10, 110, 10)
    model, test_patk, test_patk_sd, test2_patk, test2_patk_sd, test_rr, test_rr_sd, test2_rr, test2_rr_sd = learning_curve(
        model, train, test, eval_train, train2, test2, eval_train2, iterarray, item_features=item_features_concat,k=5, **{'num_threads': 4}) #num_threads 1

    
    results.append(result)
#     precisions.append(precision)
#     ranks.append(rank)
    print(time.asctime())

Mon Jun 10 14:40:55 2019
(('broadcaster', 'description_tfidf', 'title_tfidf', 'sub_tfidf'), 50)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 14:54:01 2019
Mon Jun 10 14:54:01 2019
(('broadcaster', 'genres', 'title_tfidf', 'sub_tfidf'), 51)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 15:05:02 2019
Mon Jun 10 15:05:02 2019
(('credits', 'description_tfidf', 'genres', 'title_tfidf'), 52)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 15:15:13 2019
Mon Jun 10 15:15:13 2019
(('credits', 'description_tfidf', 'genres', 'sub_tfidf'), 53)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 15:30:03 2019
Mon Jun 10 15:30:03 2019
(('credits', 'description_tfidf', 'title_tfidf', 'sub_tfidf'), 54)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 15:45:35 2019
Mon Jun 10 15:45:35 2019
(('credits', 'genres', 'title_tfidf', 'sub_tfidf'), 55)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 15:58:56 2019
Mon Jun 10 15:58:56 2019
(('description_tfidf', 'genres', 'title_tfidf', 'sub_tfidf'), 56)
10
20
30
40
50
60
70
80
90
100
Mon Jun 10 1

In [24]:
pd.DataFrame(results).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263
0,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000,80.000000,90.000000,100.000000,NaN
1,0.076455,0.093186,0.102310,0.106454,0.109416,0.112468,0.115642,0.117609,0.119440,0.120892,NaN,0.083220,0.097703,0.101937,0.107457,0.111427,0.113477,0.117429,0.119196,0.120969,0.121978,NaN,0.067896,0.080939,0.090995,0.097144,0.101577,0.104083,0.105214,0.107174,0.108819,0.109635,NaN,0.072554,0.085790,0.094201,0.098493,0.102348,0.104186,0.107257,0.108902,0.110836,0.113535,NaN,0.051300,0.052482,0.054866,0.059370,0.063475,0.064600,0.069046,0.070145,0.072452,0.075035,NaN,0.075683,0.088091,0.094567,0.098654,0.103132,0.105009,0.107

In [26]:
len(results)/11

24

In [27]:
spark.createDataFrame(pd.DataFrame(results)).write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/results/marchv2results40-64")
# spark.createDataFrame(pd.DataFrame(precisions)).write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/results/marchv2precisions22-30")
# spark.createDataFrame(pd.DataFrame(ranks)).write.parquet("gs://dataproc-jupyter-eileen.npo-data.nl/data/results/marchv2ranks22-30")

In [202]:
pd.DataFrame(combi).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,None,"(broadcaster,)","(credits,)","(description_tfidf,)","(genres,)","(title_tfidf,)","(sub_tfidf,)","(broadcaster, credits)","(broadcaster, description_tfidf)","(broadcaster, genres)","(broadcaster, title_tfidf)","(broadcaster, sub_tfidf)","(credits, description_tfidf)","(credits, genres)","(credits, title_tfidf)","(credits, sub_tfidf)","(description_tfidf, genres)","(description_tfidf, title_tfidf)","(description_tfidf, sub_tfidf)","(genres, title_tfidf)","(genres, sub_tfidf)","(title_tfidf, sub_tfidf)","(broadcaster, credits, description_tfidf)","(broadcaster, credits, genres)","(broadcaster, credits, title_tfidf)","(broadcaster, credits, sub_tfidf)","(broadcaster, description_tfidf, genres)","(broadcaster, description_tfidf, title_tfidf)","(broadcaster, description_tfidf, sub_tfidf)","(broadcaster, genres, title_tfidf)","(broadcaster, genres, sub_tfidf)","(broadcaster, title_tfidf, sub_tfidf)","(credits, description_tfidf, genres)","(credits, description_tfidf, title_tfidf)","(credits, description_tfidf, sub_tfidf)","(credits, genres, title_tfidf)","(credits, genres, sub_tfidf)","(credits, title_tfidf, sub_tfidf)","(description_tfidf, genres, title_tfidf)","(description_tfidf, genres, sub_tfidf)","(description_tfidf, title_tfidf, sub_tfidf)","(genres, title_tfidf, sub_tfidf)","(broadcaster, credits, description_tfidf, genres)","(broadcaster, credits, description_tfidf, titl...","(broadcaster, credits, description_tfidf, sub_...","(broadcaster, credits, genres, title_tfidf)","(broadcaster, credits, genres, sub_tfidf)","(broadcaster, credits, title_tfidf, sub_tfidf)","(broadcaster, description_tfidf, genres, title...","(broadcaster, description_tfidf, genres, sub_t...","(broadcaster, description_tfidf, title_tfidf, ...","(broadcaster, genres, title_tfidf, sub_tfidf)","(credits, description_tfidf, genres, title_tfidf)","(credits, description_tfidf, genres, sub_tfidf)","(credits, description_tfidf, title_tfidf, sub_...","(credits, genres, title_tfidf, sub_tfidf)","(description_tfidf, genres, title_tfidf, sub_t...","(broadcaster, credits, description_tfidf, genr...","(broadcaster, credits, description_tfidf, genr...","(broadcaster, credits, description_tfidf, titl...","(broadcaster, credits, genres, title_tfidf, su...","(broadcaster, description_tfidf, genres, title...","(credits, description_tfidf, genres, title_tfi...","(broadcaster, credits, description_tfidf, genr..."


## Optimizing model
https://github.com/lyst/lightfm/issues/286 Different evaluation results on two runs with random state set due to multiple threads

In [24]:
#feature = combi[16] # ('description_tfidf', 'genres')
feature = combi[50] # ('broadcaster', 'description_tfidf', 'title_tfidf', 'sub_tfidf')

paths = []
for i in range(0, len(feature)):
    paths.append('gs://dataproc-jupyter-eileen.npo-data.nl/data/content_features/' + feature[i])
sideinfo = spark.read.parquet(*paths)

sideinfo = sideinfo.toPandas()
sideinfo['value'] = sideinfo.value.str.encode('utf-8')
sideinfo = sideinfo[sideinfo['mid'].isin(df_lim.mid.tolist())]
item_features_concat = get_itemfeatures(sideinfo)

In [16]:
def objective_wsideinfo(params):
    epochs, learning_rate,\
    no_components, item_alpha,\
    scale = params
    
    user_alpha = item_alpha * scale
    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train, epochs=epochs,
              item_features=item_features_concat,
              num_threads=4, verbose=True)
    
    patks = lightfm.evaluation.precision_at_k(model, test,
                                              item_features=item_features_concat,
                                              train_interactions=None,
                                              k=5, num_threads=3)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Weird shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [18]:
# Grid search
space = [(1, 120), # epochs
         (10**-3, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-5, 10**-3, 'log-uniform'), # item_alpha
         (0.001, 1., 'log-uniform') # user_scaling
        ]
item_features = item_features_concat.astype(np.float32)
res_fm_itemfeat_precision = forest_minimize(objective_wsideinfo, space, n_calls=50,
                                  random_state=0,
                                  verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 49.3514
Function value obtained: -0.0432
Current minimum: -0.0432
Iteration No: 2 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 

Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 59.5264
Function value obtained: -0.1397
Current minimum: -0.1806
Iteration No: 13 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57

Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 76.5149
Function value obtained: -0.1018
Current minimum: -0.1847
Iteration No: 20 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64


Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Iteration No: 26 ended. Search finished for the next optimal point.
Time taken: 77.8553
Function value obtained: -0.1859
Current minimum: -0.1901
Iteration No: 27 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57

Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 54.4938
Function value obtained: -0.1843
Current minimum: -0.1901
Iteration No: 35 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59

Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 81.6203
Function value obtained: -0.1830
Current minimum: -0.1901
Iteration No: 45 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16

In [19]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm_itemfeat_precision.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'item_alpha', 'scaling']
for (p, x_) in zip(params, res_fm_itemfeat_precision.x):
    print('{}: {}'.format(p, x_))
print('Standard deviation: ' + str(np.std(-res_fm_itemfeat_precision.func_vals)))

Maximimum p@k found: 0.19006
Optimal parameters:
epochs: 119
learning_rate: 0.0950077507918
no_components: 87
item_alpha: 1.40170452425e-05
scaling: 0.00129052277604
Standard deviation: 0.05562915


### Train using optimal parameters

In [27]:
## Gemiddeld online

# epochs = 94
# learning_rate = 0.003
# no_components = 155
# item_alpha = 0.0008
# scale = 0.881

In [27]:
## Model1_V2

# epochs = 109
# learning_rate = 0.0145931111183
# no_components =  186
# item_alpha = 0.000458915607758
# scale = 0.451807286244

epochs = 105
learning_rate = 0.017517368561
no_components =  183
item_alpha = 0.00086611936337
scale = 0.837505047238

In [28]:
# epochs, learning_rate,\
# no_components, item_alpha,\
# scale = res_fm_itemfeat_precision.x

user_alpha = item_alpha * scale
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=learning_rate,
                no_components=no_components,
                user_alpha=user_alpha,
                item_alpha=item_alpha)
model.fit(interactions, epochs=epochs,
          item_features=item_features_concat,
          num_threads=4)

In [29]:
precision_test = lightfm.evaluation.precision_at_k(model, test, item_features=item_features_concat, train_interactions=None, k=5)
rank_test = lightfm.evaluation.reciprocal_rank(model, test, item_features=item_features_concat, train_interactions=None)

print("Precision mean: " + str(np.mean(precision_test)))
print("Precision std: " + str(np.std(precision_test)))
print("Rank mean: " + str(np.mean(rank_test)))
print("Rank std: " + str(np.std(rank_test)))

Precision mean: 0.1918399
Precision std: 0.11441082
Rank mean: 0.5132041
Rank std: 0.33908904


In [30]:
import pickle
!pip install google-cloud-storage
from google.cloud import storage
import re

def _make_gcs_uri(bucket_name, object_name):
    return "gs://{}/{}".format(bucket_name, object_name)

def pickle_and_upload(obj, bucket_name, object_name):
    """Returns the object's GCS uri."""
    print("pickling data")
    pickle_str = pickle.dumps(obj)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(object_name)
    print("uploading object {} to bucket {}".format(object_name, bucket_name))
    blob.upload_from_string(pickle_str)

    return _make_gcs_uri(bucket_name, object_name)

def split_uri(gcs_uri):
    """Splits gs://bucket_name/object_name to (bucket_name, object_name)"""
    pattern = r"gs://([^/]+)/(.+)"
    match = re.match(pattern, gcs_uri)

    bucket_name = match.group(1)
    object_name = match.group(2)

    return bucket_name, object_name

def download_and_unpickle(bucket_name, object_name):
    blob = get_blob(bucket_name, object_name)
    pickle_str = blob.download_as_string()

    obj = pickle.loads(pickle_str)
    return obj

def get_blob(bucket_name, object_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(object_name)
    return blob


def get_uri_blob(gcs_uri):
    bucket_name, object_name = split_uri(gcs_uri)
    return get_blob(bucket_name, object_name)

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
ERROR: google-cloud-bigquery 1.14.0 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.28.1 which is incompatible.
ERROR: google-cloud-bigtable 0.33.0 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.28.1 which is incompatible.
  Found existing installation: google-cloud-core 1.0.1
    Uninstalling google-cloud-core-1.0.1:
      Successfully uninstalled google-cloud-core-1.0.1


In [31]:
obj = model
obj_name = split_uri('gs://dataproc-jupyter-eileen.npo-data.nl/data/results/model2_v2')
pickle_and_upload(obj, obj_name[0], obj_name[1])

pickling data
uploading object data/results/model2_v2 to bucket dataproc-jupyter-eileen.npo-data.nl


'gs://dataproc-jupyter-eileen.npo-data.nl/data/results/model2_v2'

## List creation
Now you see the recommended lists <br>
(can also show a few items which were also chosen - known_positives) <br>
https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b

In [ ]:
# None: de luizenmoeder, wie is de mol, 2 voor 12, oogappels, met het mes op tafel
# broadcaster: wie is de mol, oogappels, moltalk, verborgen verleden, floortje naar het einde van de wereld

In [56]:
obj_name = split_uri('gs://dataproc-jupyter-eileen.npo-data.nl/data/results/model4')
model0 = download_and_unpickle(obj_name[0], obj_name[1])

In [45]:
# Also, consider filtering out items already seen (because that isn't happening now)

# show recommendations for user
user = user_list[21]
print(idx_to_uid.get(user))
scores = model.predict(np.int32(user, dtype=np.int32), item_list) #31
score_df = pd.DataFrame({'itemlist' : item_list, 'scores' : scores})
score_df = score_df.drop_duplicates()
score_df = score_df.sort_values('scores', ascending=False)
series_ref = []
for x in score_df['itemlist'].tolist():
    series_ref.append(idx_to_mid.get(x))
score_df['seriesRef'] = series_ref
score_df

b2421249-a92d-4501-95ce-35fedcd12352


,itemlist,scores,seriesRef
12391,2598,0.575958,POMS_S_NPO_4780015
9794,2492,0.561227,AT_2048191
948,2500,0.517884,POW_03825084
3897,2528,0.440839,POW_00828626
419,2597,0.431538,VPWON_1275436
4459,2516,0.430931,POMS_S_NPO_3602337
1418,2509,0.378966,POMS_S_NPO_2966850
40815,2599,0.376753,POMS_S_EO_690753
6955,2594,0.362682,POW_02977467
1424,2622,0.362659,POMS_S_BNN_6114668


In [46]:
df_int[df_int['uid'] == 'b2421249-a92d-4501-95ce-35fedcd12352'] 
#floortje naar het einde van de wereld, de wereld draait door, per seconde wijzer

,month,day,uid,mid
312018,3,17,b2421249-a92d-4501-95ce-35fedcd12352,POW_03804985
878899,3,19,b2421249-a92d-4501-95ce-35fedcd12352,POW_03164058
897368,3,22,b2421249-a92d-4501-95ce-35fedcd12352,POW_03164058
976988,3,16,b2421249-a92d-4501-95ce-35fedcd12352,POW_03804985
1072729,3,18,b2421249-a92d-4501-95ce-35fedcd12352,POW_03804985
1140639,3,22,b2421249-a92d-4501-95ce-35fedcd12352,POW_00823240
1207932,3,20,b2421249-a92d-4501-95ce-35fedcd12352,POW_03164058
1209712,3,21,b2421249-a92d-4501-95ce-35fedcd12352,POW_03164058


In [50]:
sideinfo[sideinfo['mid'].isin(['POW_03804985', 'POW_03164058', 'POW_00823240'])].sort_values('value')

,mid,value,feature
58513,POW_03164058,KRNC,broadcaster
55081,POW_03804985,KRNC,broadcaster
58496,POW_00823240,KRNC,broadcaster
57433,POW_00823240,KRO,broadcaster
39557,POW_03164058,albert,description
21675,POW_03804985,cadi,description
12091,POW_00823240,dood,description
21670,POW_03804985,dylan,description
39564,POW_03164058,forbes,description
12086,POW_00823240,gevonden,description
